In [2]:
import fiona, rasterio
import cv2, sys, copy, scipy, torch, torchvision

import numpy as np
import seaborn as sns
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision.models as models

from tqdm import tqdm
from shapely import geometry
from datetime import datetime
from rasterio.mask import mask 
from extractor_helper import extractor
from IPython.display import clear_output
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter

np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)

In [4]:
sat_image = rasterio.open("data/1030010056130F00_MS_Pan_modified_KNN.tif")
sat_points = fiona.open("data/AB_points.shp", "r")

In [7]:
parchsize = 5
patch, coordinates, _ = extractor(sat_image ,sat_points, size=parchsize, normalize=True)

In [20]:
class mydata(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return torch.tensor(self.data[idx]).permute(2,0,1).float()[None]


In [21]:
data = mydata(patch)

In [23]:
for i in data: print(i.shape);break

torch.Size([1, 8, 10, 10])


In [30]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

model1 = models.resnet18(pretrained=True)
model2 = models.resnet18(pretrained=True)
# model3 = models.resnet34(pretrained=True)
model4 = models.resnet50(pretrained=True)
model5 = models.wide_resnet50_2(pretrained=True)




###### CHANGE Classifier 
model1.conv1 = nn.Conv2d(8,64, kernel_size=(7,7), stride=(2, 2), padding=(3, 3), bias=False)
model1.fc = nn.Linear(512,4)

model2.conv1 = nn.Conv2d(8,64, kernel_size=(7,7), stride=(2, 2), padding=(3, 3), bias=False)
model2.fc = nn.Linear(512,4)

# model3.conv1 = nn.Conv2d(8,64, kernel_size=(7,7), stride=(2, 2), padding=(3, 3), bias=False)
# model3.fc = nn.Linear(512,4)

model4.conv1 = nn.Conv2d(8,64, kernel_size=(7,7), stride=(2, 2), padding=(3, 3), bias=False)
model4.fc = nn.Sequential(nn.Dropout(p=0.5),
                         nn.Linear(2048,1000),
                         nn.BatchNorm1d(1000),
                         nn.ReLU(),
                         nn.Dropout(0.5),
                         nn.Linear(1000, 4))

model5.conv1 = nn.Conv2d(8,64, kernel_size=(7,7), stride=(2, 2), padding=(3, 3), bias=False)
model5.fc = nn.Sequential(nn.Dropout(p=0.5),
                         nn.Linear(2048,1000),
                         nn.BatchNorm1d(1000),
                         nn.ReLU(),
                         nn.Dropout(0.5),
                         nn.Linear(1000, 4))


model1.load_state_dict(torch.load('./best_ensumble/resnet18_80.pth'))
model2.load_state_dict(torch.load('./best_ensumble/resnet18_84.pth'))
model4.load_state_dict(torch.load('./best_ensumble/resnet50_82.pth'))
model5.load_state_dict(torch.load('./best_ensumble/wide_resnet50_80.pth'))

<All keys matched successfully>

In [36]:
models = [model1, model2, model4, model5]

In [60]:
outputs = np.empty((len(data), len(models)))
with torch.no_grad():
    for i, model in enumerate(models):
        model.eval(), model.to(device)
        for j,instance in enumerate(data):
            outputs[j,i] = torch.argmax(model(instance.to(device))).item()

In [61]:
outputs

array([[0., 0., 1., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.],
       ...,
       [0., 1., 1., 1.],
       [3., 1., 0., 1.],
       [0., 0., 1., 1.]])

In [62]:
for i in outputs: 
    if 2.0 in i:
        print(i)

[3. 1. 2. 1.]
[0. 0. 2. 1.]
[0. 1. 2. 1.]
[1. 1. 2. 1.]
[3. 1. 2. 1.]


In [63]:
for i in outputs: 
    if 0.0 in i:
        print(i)

[0. 0. 1. 1.]
[0. 0. 1. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 1.]
[0. 0. 1. 0.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 0.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 3.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 0.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 0.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 3.]
[0. 1. 0. 0.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 0.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 0.]
[3. 0. 0. 3.]
[0. 0. 1. 1.]
[0. 0. 1. 0.]
[0. 0. 0. 0.]
[0. 0. 1. 0.]
[3. 0. 0. 3.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 0.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 0.]
[0. 0. 0. 0.]
[0. 0. 1. 1.]
[0. 0.

[0. 0. 0. 0.]
[3. 3. 0. 1.]
[0. 3. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 1. 1. 1.]
[0. 0. 2. 1.]
[0. 1. 0. 1.]
[0. 1. 1. 1.]
[0. 0. 1. 1.]
[1. 1. 0. 1.]
[0. 1. 1. 1.]
[0. 0. 1. 1.]
[0. 1. 1. 1.]
[0. 0. 0. 1.]
[0. 1. 0. 1.]
[0. 1. 1. 1.]
[0. 0. 1. 1.]
[0. 1. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 3. 0. 1.]
[0. 0. 1. 1.]
[3. 1. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[3. 1. 1. 0.]
[0. 1. 1. 1.]
[0. 1. 0. 1.]
[0. 1. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 0.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 1. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[3. 1. 0. 1.]
[0. 1. 1. 1.]
[3. 1. 0. 1.]
[0. 3. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 1. 1. 1.]
[0. 0. 0. 3.]
[0. 1. 1. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[3. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 0.

[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 0. 1. 0.]
[0. 0. 0. 1.]
[0. 0. 0. 0.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 3.]
[0. 1. 1. 1.]
[0. 0. 1. 0.]
[0. 0. 0. 1.]
[3. 1. 0. 1.]
[0. 0. 1. 0.]
[0. 0. 1. 3.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 0.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[3. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 0.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 0.]
[0. 0. 1. 1.]
[0. 0. 1. 0.]
[0. 0. 1. 0.]
[0. 0. 1. 0.]
[0. 1. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[3. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 3.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 0.]
[0. 0. 1. 1.]
[0. 0. 0. 0.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 1. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 0.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 1. 1. 3.]
[0. 1. 0. 1.]
[0. 0. 0. 1.]
[0. 0.

[0. 0. 1. 1.]
[0. 0. 0. 3.]
[0. 0. 1. 0.]
[0. 0. 0. 1.]
[0. 0. 0. 0.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 0.]
[0. 0. 0. 1.]
[0. 0. 0. 3.]
[0. 0. 0. 1.]
[0. 0. 1. 0.]
[0. 0. 1. 0.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 3.]
[0. 0. 1. 0.]
[0. 0. 1. 1.]
[0. 0. 0. 0.]
[0. 0. 1. 0.]
[0. 0. 1. 0.]
[0. 0. 1. 0.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 0.]
[0. 0. 1. 1.]
[0. 0. 1. 3.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 0.]
[0. 0. 1. 1.]
[0. 0. 1. 0.]
[0. 0. 1. 3.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 3.]
[0. 0. 1. 3.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 0.]
[0. 0. 0. 3.]
[0. 0. 1. 0.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 1. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 0.]
[0. 0. 0. 3.]
[0. 0. 0. 1.]
[0. 0. 1. 0.]
[0. 0. 0. 1.]
[0. 0.

[0. 1. 1. 1.]
[0. 0. 1. 0.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 0.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 0.]
[0. 0. 0. 1.]
[0. 0. 1. 0.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 0.]
[0. 0. 1. 1.]
[0. 0. 0. 0.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 0.]
[0. 0. 0. 3.]
[0. 0. 1. 0.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 0. 1. 0.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 0.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 0.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 0. 1. 3.]
[0. 0. 1. 3.]
[0. 0. 1. 1.]
[0. 0. 1. 0.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 0.]
[0. 0. 1. 0.]
[0. 0. 1. 0.]
[0. 0. 1. 1.]
[0. 0. 1. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 1. 1.]
[0. 0. 1. 0.]
[0. 0.